# Subsea Inpanting using FGVC ([paper](https://arxiv.org/abs/2009.01835), [code](https://github.com/vt-vl-lab/FGVC))
 - It will be necessary to allow Internet Access at the Kaggle notebook settings
 - It will be necessary to activate GPU accelerator at the Kaggle notebook settings 

# 1. Setup

## 1.1. Install [viajen](https://github.com/brunomsantiago/viajen) for animation visualization

In [ ]:
!pip install viajen

## 1.2. Install imageio-ffmpeg

In [ ]:
!pip install imageio-ffmpeg

## 1.3. Download FGVC code and models from [subsea inpainting repository](https://github.com/brunomsantiago/subsea_inpainting)

In [ ]:
!git clone https://github.com/brunomsantiago/subsea_inpainting --branch models

## 1.4. Normal imports

In [ ]:
# Standard Library imports
from pathlib import Path
import time
import sys

# Third party imports
import cv2
from IPython.core.display import HTML
import numpy as np
from PIL import Image
from viajen import animate

## 1.5. Local import (non-installed module)

In [ ]:
sys.path.insert(1, './subsea_inpainting/code/FGVC/tool')
from video_completion import video_completion_seamless

## 1.6. Custom dictionary to emulate argparse results

In [ ]:
class AttributeDict(dict):
    '''
    Dict which keys can be accessed like a normal dict:
        my_dict['key']
    or like a class:
        my_dict.key
    Useful to mimic the behaviour of argparse results
    '''
    def __getattr__(self, attr):
        return self[attr]
    def __setattr__(self, attr, value):
        self[attr] = value
    def _get_kwargs(self):
        return sorted(self.__dict__.items())

# 2. Frames to be inpainted (+ masks)

## 2.1. Select clip to work with

In [ ]:
data_name = '01a'
# data_name = '01b'
# data_name = '01c'
# data_name = '02a'
# data_name = '03a'
# data_name = '03b'
# data_name = '04a'
# data_name = '05a'
# data_name = '05b'
# data_name = '05c'
# data_name = '05d'
# data_name = '05e'
# data_name = '05f'

## 2.2. Mount folders

In [ ]:
base_input_folder = '../input/subsea-inpainting-dataset/'
base_output_folder = './'

frames_folder = Path(base_input_folder) / 'prepared_images' / data_name / 'frames'
masks_folder = Path(base_input_folder) / 'prepared_images' / data_name / 'masks'

results_folder = Path(base_output_folder) / 'results'/ 'fgvc' / data_name

## 2.3. View clip

In [ ]:
animate.folder(frames_folder, max_height=200)

In [ ]:
animate.folder(masks_folder, max_height=200)

# 3. Inpaiting

### 3.1. Args for video completion

In [ ]:
args = AttributeDict()
# Video completion
args['seamless'] = True
args['edge_guide'] = False
args['mode'] = 'object_removal'
args['path'] = frames_folder
args['path_mask'] = masks_folder
args['outroot'] = results_folder
args['consistencyThres'] = np.inf
args['alpha'] = 0.1
args['Nonlocal'] = False
# RAFT
args['model'] ='./subsea_inpainting/models_weights/fgvc/raft-things.pth'
args['small'] = False
args['mixed_precision'] = False
args['alternate_corr'] = False
# Deepfill
args['deepfill_model'] = './subsea_inpainting/models_weights/fgvc/imagenet_deepfill.pth'
# Edge completion
args['edge_completion_model'] = './subsea_inpainting/models_weights/fgvc/edge_completion.pth'

## 3.2. Function to apply inpainting

In [ ]:
def inpaint_fgvc(args):
    start_time = time.time()     
    video_completion_seamless(args)
    return time.time() - start_time

## 3.3 Execute inpainting
 - May take a long time

In [ ]:
execution_time = inpaint_fgvc(args)
print(f'\n\n--- Execution time: {execution_time:.2f} seconds ---')

# 4. Show results

In [ ]:
display(HTML('<h2>Original Frames<h2>'))
display(animate.folder(frames_folder))

display(HTML(f'<br><br><h2>Inpainted Frames ({execution_time:.1f} seconds processing)<h2>'))
display(animate.folder(results_folder / 'frame_seamless_comp_final'))